Se importan las librerias

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [ ]:
df_test=pd.read_csv("..\datasets_original\hospitalizaciones_test.csv",sep=",")
df_train=pd.read_csv("..\datasets_original\hospitalizaciones_train.csv",sep=",")
df_test["Stay (in days)"]=np.nan

In [ ]:
df_train['Stay (in days)'] = np.where(df_train['Stay (in days)']<8, 0, 1)

Se cargan los .csv previamente modificados y se crean los dataframe test y train.

In [2]:
df_test=pd.read_csv("data_cleansing\df_test_mod.csv",sep=",")
df_train=pd.read_csv("data_cleansing\df_train_mod.csv",sep=",")

Debido a la gran cantidad de columnas, en lugar de usar un grafico se opto por imprimir las 4 columnas con mayor y menor correlacion con la columna "Stay (in days)"

In [3]:
print(df_train.corr().nlargest(5,"Stay (in days)")["Stay (in days)"].append(df_train.corr().nsmallest(4,"Stay (in days)")["Stay (in days)"]))

Stay (in days)             1.000000
doctor_name_Dr Mark        0.133455
Department_anesthesia      0.130680
Department_gynecology      0.105449
Age_21-30                  0.100590
Department_radiotherapy   -0.288060
doctor_name_Dr Sam        -0.237977
Age_31-40                 -0.169099
Age_41-50                 -0.132071
Name: Stay (in days), dtype: float64


C:\Users\Lautaro\AppData\Local\Temp\ipykernel_12040\1740225458.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  print(df_train.corr().nlargest(5,"Stay (in days)")["Stay (in days)"].append(df_train.corr().nsmallest(4,"Stay (in days)")["Stay (in days)"]))


Se analiza la proporcion de los datos

In [3]:
corta=df_train[(df_train["Stay (in days)"]==0)]["Stay (in days)"].value_counts()[0]
larga=df_train[(df_train["Stay (in days)"]==1)]["Stay (in days)"].value_counts()[1]
total=corta+larga
print("Porcentaje estadia corta: ",(corta/total)*100)
print("Porcentaje estadia larga: ",(larga/total)*100)

Porcentaje estadia corta:  14.971219512195121
Porcentaje estadia larga:  85.02878048780488


Se importan las librerias de scikit-learn

In [3]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,recall_score
import sklearn.metrics as metrics
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier #Modelo 1
from sklearn.neighbors import KNeighborsClassifier #Modelo 2
from sklearn.linear_model import LogisticRegression #Metamodelo

Se arma el test_split usando df_train y 30% de esos datos para testeo

In [4]:
y=df_train["Stay (in days)"].values.reshape(-1, 1)
#Se toman solo las columnas necesarias
X=df_train[["doctor_name_Dr Mark","Department_anesthesia","Department_radiotherapy","doctor_name_Dr Sam","Age_31-40"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)


Usando GridSearchCV se obtienen los mejores parametros para el modelo de random forest.

In [18]:
grid = { 
    'n_estimators': [100,200,300],
    'max_depth' : [1,2,3,4],
    'random_state' : [42]
}
random_forest_grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid, cv= 5,n_jobs=-1)
random_forest_grid.fit(X_train, y_train)


c:\Users\Lautaro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4],
                         'n_estimators': [100, 200, 300],
                         'random_state': [42]})

In [6]:
#Estos son los mejores parametros para el modelo
print(random_forest_grid.best_params_)

NameError: name 'random_forest_grid' is not defined

Se configura el modelo usando los parametros obtenidos con GridSearchCV

In [9]:
random_forest = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42,n_jobs=-1, oob_score = True) 
random_forest.fit(X_train, y_train)

C:\Users\Lautaro\AppData\Local\Temp\ipykernel_12824\2813575904.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest.fit(X_train, y_train)


RandomForestClassifier(max_depth=2, n_jobs=-1, oob_score=True, random_state=42)

El modelo se entrena usando los datos armados con train_test_split

In [10]:
pred = random_forest.predict(X_test)
y_train_pred = random_forest.predict(X_train)
y_test_pred = random_forest.predict(X_test)

print("Accuracy para y_train:",metrics.accuracy_score(y_train, y_train_pred))
print("Recall para y_train:",metrics.recall_score(y_train, y_train_pred))

print("Accuracy para y_test:",metrics.accuracy_score(y_test, y_test_pred))
print("Recall para y_test:",metrics.recall_score(y_test, y_test_pred))

Accuracy para y_train: 0.888609756097561
Recall para y_train: 0.9993279597431495
Accuracy para y_test: 0.8881300813008131
Recall para y_test: 0.9993976191614476


Una vez entrenado el modelo se utilizando los datos del dataframe df_test para crear una columna llamada pred_1 que se va a completar con las predicciones del modelo.

In [11]:
#Se separan las columnas necesarias de df_test para su testeo con el modelo entrenado
test_model=df_test[["doctor_name_Dr Mark","Department_anesthesia","Department_radiotherapy","doctor_name_Dr Sam","Age_31-40"]]

#Se agrega la columna "pred_1" con las predicciones del modelo
pred_model_1=random_forest.predict(test_model)
df_test["pred_1"]=pred_model_1

df_test[["doctor_name_Dr Mark","Department_anesthesia","Department_radiotherapy","doctor_name_Dr Sam","Age_31-40","pred_1"]].sample(5)

,doctor_name_Dr Mark,Department_anesthesia,Department_radiotherapy,doctor_name_Dr Sam,Age_31-40,pred_1
80010,-0.327841,-0.311432,2.220514,-0.371973,-0.593395,1
119389,-0.327841,-0.311432,-0.450346,-0.371973,-0.593395,1
235977,3.050262,3.210973,-0.450346,-0.371973,-0.593395,1
370272,-0.327841,-0.311432,-0.450346,-0.371973,-0.593395,1
352001,-0.327841,-0.311432,-0.450346,-0.371973,1.685218,1


Se crea un nuevo modelo usando KNeighborsClassifier.
En este caso no se uso GridSearchCV debido a la cantidad elevada de tiempo que tarda obtener los resultados, por lo que solo se probaron 2 configuraciones distintas y se eligio la que mejor Recall y Acurracy producia.

In [7]:
# Se define el clasificador
knn = KNeighborsClassifier(7,n_jobs=-1)

# Se entrena el modelo
knn.fit(X_train, y_train)

print("Accuracy para y_train:",metrics.accuracy_score(y_train, knn.predict(X_train)))
print("Recall para y_train:",metrics.recall_score(y_train, knn.predict(X_train)))

print("Accuracy para y_test:",metrics.accuracy_score(y_test, knn.predict(X_test)))
print("Recall para y_test:",metrics.recall_score(y_test, knn.predict(X_test)))

c:\Users\Lautaro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neighbors\_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Accuracy para y_train: 0.888609756097561
Recall para y_train: 0.9993279597431495
Accuracy para y_test: 0.8881300813008131
Recall para y_test: 0.9993976191614476


In [12]:
pred_model_2=knn.predict(test_model)
df_test["pred_2"]=pred_model_2


In [19]:
df_test[["doctor_name_Dr Mark","Department_anesthesia","Department_radiotherapy","doctor_name_Dr Sam","Age_31-40","pred_1","pred_2"]].sample(5)

,doctor_name_Dr Mark,Department_anesthesia,Department_radiotherapy,doctor_name_Dr Sam,Age_31-40,pred_1,pred_2
84435,-0.327841,-0.311432,2.220514,-0.371973,-0.593395,1,1
237149,-0.327841,-0.311432,-0.450346,-0.371973,-0.593395,1,1
295261,-0.327841,-0.311432,-0.450346,-0.371973,-0.593395,1,1
187251,-0.327841,-0.311432,-0.450346,-0.371973,1.685218,1,1
120387,-0.327841,-0.311432,-0.450346,-0.371973,1.685218,1,1


In [ ]:
estimator_list = [
    ('knn',knn),
    ('random_forest',random_forest)]

stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=LogisticRegression(n_jobs=-1))

# Se entrena el metamodelo
stack_model.fit(X_train, y_train)

# Desempeño del metamodelo en train, calculando la precisión (accuracy)
print("Accuracy para y_train: ",accuracy_score(y_train, stack_model.predict(X_train)))
print("Recall para y_train: ",recall_score(y_train, stack_model.predict(X_train)))

# Desempeño del metamodelo en test, calculando la precisión (accuracy)
print("Accuracy para y_test: ",accuracy_score(y_test, stack_model.predict(X_test)))
print("Recall para y_test: ",recall_score(y_test, stack_model.predict(X_test)))


In [21]:
# Desempeño del metamodelo en train, calculando la precisión (accuracy)
print("Accuracy para y_train: ",metrics.accuracy_score(y_train, stack_model.predict(X_train)))
print("Recall para y_train: ",metrics.recall_score(y_train, stack_model.predict(X_train)))

# Desempeño del metamodelo en test, calculando la precisión (accuracy)
print("Accuracy para y_test: ",metrics.accuracy_score(y_test, stack_model.predict(X_test)))
print("Recall para y_test: ",metrics.recall_score(y_test, stack_model.predict(X_test)))

Accuracy para y_train:  0.888609756097561
Recall para y_train:  0.9993279597431495
Accuracy para y_test:  0.8881300813008131
Recall para y_test:  0.9993976191614476


In [23]:
pred_model_3=stack_model.predict(test_model)
df_test["pred_3"]=pred_model_3
df_test[["doctor_name_Dr Mark","Department_anesthesia","Department_radiotherapy","doctor_name_Dr Sam","Age_31-40","pred_1","pred_2","pred_3"]].sample(5)

,doctor_name_Dr Mark,Department_anesthesia,Department_radiotherapy,doctor_name_Dr Sam,Age_31-40,pred_1,pred_2,pred_3
302073,-0.327841,-0.311432,-0.450346,-0.371973,-0.593395,1,1,1
11835,-0.327841,-0.311432,-0.450346,-0.371973,1.685218,1,1,1
21824,-0.327841,-0.311432,-0.450346,-0.371973,-0.593395,1,1,1
284878,-0.327841,-0.311432,2.220514,2.688367,-0.593395,1,1,1
4759,-0.327841,-0.311432,-0.450346,-0.371973,-0.593395,1,1,1


In [36]:
df_test["pred_final"]=round((df_test["pred_1"]+df_test["pred_2"]+df_test["pred_3"])/3,1)
df_test["pred_final"]=df_test["pred_final"].astype(int)
df_test[["pred_final","pred_1","pred_2","pred_3"]]

,pred_final,pred_1,pred_2,pred_3
0,1,1,1,1
1,1,1,1,1
2,1,1,1,1
3,1,1,1,1
4,1,1,1,1
...,...,...,...,...
409995,1,1,1,1
409996,1,1,1,1
409997,1,1,1,1
409998,1,1,1,1


In [45]:
pred=pd.DataFrame(df_test["pred_final"])
pred.rename(columns={'pred_final':'pred'},inplace=True)
pred.to_csv("Lautaro555.csv",index=False)